In [1]:
#!pip install pm4py

In [3]:
import pm4py
import pandas as pd
import numpy as np

In [4]:
log = pm4py.read_xes('Road_Traffic_Fine_Management_Process.xes')

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

In [5]:
df = pm4py.convert_to_dataframe(log)

In [6]:
df.to_csv("log.csv")

In [7]:
df.head()

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
0,35.0,561,NIL,Create Fine,A,0.0,complete,2006-07-24 00:00:00+00:00,157.0,0.0,A1,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-05 00:00:00+00:00,NaN,NaN,A1,11.0,NaN,NaN,NaN,NaN
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-02 00:00:00+00:00,157.0,0.0,A100,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-12 00:00:00+00:00,NaN,NaN,A100,11.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-01-15 00:00:00+00:00,NaN,NaN,A100,NaN,P,P,NaN,NaN


In [8]:
print(df['time:timestamp'].min())
print(df['time:timestamp'].max())
print('**************************************')
display(df.describe().T)
print('**************************************')
display(df.dtypes)
print('**************************************')
print('Missing value counts:')
df.isnull().sum()

2000-01-01 00:00:00+00:00
2013-06-18 00:00:00+00:00
**************************************


,count,mean,std,min,25%,50%,75%,max
amount,230230.0,63.681656,89.677991,0.0,33.6,38.00,71.50,8000.0
totalPaymentAmount,227971.0,17.815604,36.779382,0.0,0.0,0.00,35.00,4021.0
article,150370.0,113.246658,68.219475,7.0,7.0,157.00,157.00,401.0
points,150370.0,0.078879,0.577687,0.0,0.0,0.00,0.00,10.0
expense,103987.0,11.948984,4.023190,0.0,10.0,11.88,13.50,76.0
paymentAmount,77601.0,46.410027,39.345561,0.0,33.6,36.00,44.67,3975.0
matricola,555.0,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0


**************************************


amount                              float64
org:resource                         object
dismissal                            object
concept:name                         object
vehicleClass                         object
totalPaymentAmount                  float64
lifecycle:transition                 object
time:timestamp          datetime64[ns, UTC]
article                             float64
points                              float64
case:concept:name                    object
expense                             float64
notificationType                     object
lastSent                             object
paymentAmount                       float64
matricola                           float64
dtype: object

**************************************
Missing value counts:


amount                  331240
org:resource            410545
dismissal               406404
concept:name                 0
vehicleClass            411100
totalPaymentAmount      333499
lifecycle:transition         0
time:timestamp               0
article                 411100
points                  411100
case:concept:name            0
expense                 457483
notificationType        481610
lastSent                483241
paymentAmount           483869
matricola               560915
dtype: int64

In [10]:
df = df.rename(columns={'case:concept:name':'case_id',
                   'concept:name':'concept_name',
                   'org:resource':'resource',
                   'lifecycle:transition':'lifecycle_transition',
                   'time:timestamp':'timestamp'})

In [11]:
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)

In [12]:
df = df.sort_values(by=['case_id', 'timestamp'], ascending=True)

In [13]:
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"{col} --> {df[col].unique()} -- nunique: {df[col].nunique()}\n")

resource --> ['561' nan '537' '550' '541' '557' '558' '559' '0' '536' '560' '562' '563'
 '564' '556' '538' '565' '566' '568' '567' '9' '27' '26' '21' '28' '17'
 '10' '35' '704' '24' '12' '16' '20' '23' '30' '32' '36' '31' '33' '11'
 '29' '34' '22' '828' '817' '826' '821' '813' '829' '807' '825' '818'
 '824' '827' '819' '811' '820' '802' '823' '816' '810' '830' '808' '57'
 '836' '13' '831' '832' '833' '834' '835' '837' '838' '839' '25' '37'
 '850' '53' '54' '46' '55' '42' '39' '40' '56' '45' '49' '51' '843' '845'
 '44' '52' '43' '58' '840' '842' '841' '41' '50' '48' '844' '47' '61' '4'
 '14' '3' '849' '851' '59' '847' '857' '848' '855' '856' '852' '66' '63'
 '65' '62' '858' '846' '861' '546' '540' '548' '552' '554' '555' '553'
 '551' '38' '853' '854' '60' '64' '15' '860' '859' '862' '864' '865' '866'
 '867' '868' '869' '870' '8' '18' '19'] -- nunique: 148

dismissal --> ['NIL' nan '#' 'G' 'N' 'K' '5' '3' 'A' 'I' 'D' 'T' 'E' '@' 'M' 'Q' 'F' 'V'
 'U' 'C' 'B' '$' 'Z' 'J' 'R' '2' '4'] -- nu

# Conformance Checking

In [145]:
def check_conformation(dataframe, f_cname, s_cname, law):
    result = {'positive': [], 'negative': []}
    
    filtered_df = dataframe[dataframe['concept_name'].isin([f_cname, s_cname])]

    agg_data = (
        filtered_df.groupby(['case_id', 'concept_name'])['timestamp']
        .agg(['min', 'max'])
        .unstack()
    )
    agg_data.columns = ['_'.join(filter(None, col)) for col in agg_data.columns]

    for case_id, row in agg_data.iterrows():
        first_key = f"min_{f_cname}"
        second_key = f"max_{s_cname}"
        first_ts = row[first_key]
        second_ts = row[second_key]

        if pd.isna(first_ts) or pd.isna(second_ts):
            continue

        day_diff = (second_ts - first_ts).days

        if day_diff < 0:
            continue
        elif day_diff < law:
            result['positive'].append(case_id)
        else:
            result['negative'].append(case_id)

    return result

def return_percent(result, metric_one, metric_two, days):
    total_cases = len(result['positive']) + len(result['negative'])
    if total_cases == 0:
        print("No cases to evaluate.")
        return
        
    p_ratio = (len(result['positive']) / total_cases) * 100
    n_ratio = (len(result['negative']) / total_cases) * 100

    print(f'***** {metric_one}(before) & {metric_two}(after) *****\n')
    print(f'{p_ratio:.1f}% of the cases conform with the time constraints imposed by the law. (less than {days} days)')
    print(f'{n_ratio:.1f}% of the cases fail to conform with the time constraints imposed by the law.')
    print(f'Total number of cases inspected: {total_cases}')

def find_average_time(dataframe, f_cname, s_cname, law):
    result = {'positive': [], 'negative': []}
    
    filtered_df = dataframe[dataframe['concept_name'].isin([f_cname, s_cname])]

    agg_data = (
        filtered_df.groupby(['case_id', 'concept_name'])['timestamp']
        .agg(['min', 'max'])
        .unstack()
    )
    agg_data.columns = ['_'.join(filter(None, col)) for col in agg_data.columns]

    for case_id, row in agg_data.iterrows():
        first_key = f"min_{f_cname}"
        second_key = f"max_{s_cname}"
        first_ts = row[first_key]
        second_ts = row[second_key]

        if pd.isna(first_ts) or pd.isna(second_ts):
            continue

        day_diff = (second_ts - first_ts).days
            
        if day_diff < 0:
            continue
        elif day_diff < law:
            result['positive'].append(day_diff)
        else:
            result['negative'].append(day_diff)

    return result

def calculate_stats(store, f_cname, s_cname, law):
    neg_mean, neg_med = np.mean(store['negative']), np.median(store['negative'])
    pos_mean, pos_med = np.mean(store['positive']), np.median(store['positive'])
    print(f'***** {f_cname}(before) & {s_cname}(after) ***** Limit: {law} days\n')
    print(f"1.1) Mean day difference for negative cases: {neg_mean:.1f}")
    print(f"1.2) Median day difference for negative cases: {neg_med:.1f}\n")
    print(f"2.1) Mean day difference for positive cases: {pos_mean:.1f}")
    print(f"2.2) Median day difference for positive cases: {pos_med:.1f}\n")
    print(f"3.1) Mean day difference for all cases: {(neg_mean+pos_mean)/2:.1f}")
    print(f"3.2) Median day difference for all cases: {(neg_med+pos_med)/2:.1f}")

In [147]:
#(
#        df.loc[df.concept_name.isin(['Create Fine', 'Send Fine']), :].groupby(['case_id', 'concept_name'])['timestamp']
#        .agg(['min', 'max'])
#       .unstack()
#    )

In [16]:
#Since we have lots of rows, computation time is high. Instead, we can randomly sample case IDs to decrease the dataset size.
#sampled_cases = np.random.choice(df.case_id, size=10, replace=False)
#df_sampled = df.loc[df.case_id.isin(sampled_cases), :]

#no need to do that for now

In [57]:
conformation_results = check_conformation(df, 'Create Fine', 'Send Fine', 90)
return_percent(conformation_results, 'Create Fine', 'Send Fine', 90)

***** Create Fine(before) & Send Fine(after) *****

51.4% of the cases conform with the time constraints imposed by the law. (less than 90 days)
48.6% of the cases fail to conform with the time constraints imposed by the law.
Total number of cases inspected: 103987


In [59]:
conformation_results = check_conformation(df, 'Insert Fine Notification', 'Appeal to Judge', 60)
return_percent(conformation_results, 'Insert Fine Notification', 'Appeal to Judge', 60)

***** Insert Fine Notification(before) & Appeal to Judge(after) *****

55.2% of the cases conform with the time constraints imposed by the law. (less than 60 days)
44.8% of the cases fail to conform with the time constraints imposed by the law.
Total number of cases inspected: 538


In [61]:
conformation_results = check_conformation(df, 'Insert Fine Notification', 'Insert Date Appeal to Prefecture', 60)
return_percent(conformation_results, 'Insert Fine Notification', 'Insert Date Appeal to Prefecture', 60)

***** Insert Fine Notification(before) & Insert Date Appeal to Prefecture(after) *****

83.6% of the cases conform with the time constraints imposed by the law. (less than 60 days)
16.4% of the cases fail to conform with the time constraints imposed by the law.
Total number of cases inspected: 4015


### Average Day Differences between Activities:

In [165]:
store_diffs = find_average_time(df, 'Create Fine', 'Send Fine', 90)
calculate_stats(store_diffs, 'Create Fine', 'Send Fine', 90)

***** Create Fine(before) & Send Fine(after) ***** Limit: 90 days

1.1) Mean day difference for negative cases: 123.4
1.2) Median day difference for negative cases: 122.0

2.1) Mean day difference for positive cases: 53.5
2.2) Median day difference for positive cases: 54.0

3.1) Mean day difference for all cases: 88.5
3.2) Median day difference for all cases: 88.0


In [166]:
store_diffs = find_average_time(df, 'Insert Fine Notification', 'Appeal to Judge', 60)
calculate_stats(store_diffs, 'Insert Fine Notification', 'Appeal to Judge', 60)

***** Insert Fine Notification(before) & Appeal to Judge(after) ***** Limit: 60 days

1.1) Mean day difference for negative cases: 177.0
1.2) Median day difference for negative cases: 157.0

2.1) Mean day difference for positive cases: 30.1
2.2) Median day difference for positive cases: 30.0

3.1) Mean day difference for all cases: 103.5
3.2) Median day difference for all cases: 93.5


In [167]:
store_diffs = find_average_time(df, 'Insert Fine Notification', 'Insert Date Appeal to Prefecture', 60)
calculate_stats(store_diffs, 'Insert Fine Notification', 'Insert Date Appeal to Prefecture', 60)

***** Insert Fine Notification(before) & Insert Date Appeal to Prefecture(after) ***** Limit: 60 days

1.1) Mean day difference for negative cases: 97.9
1.2) Median day difference for negative cases: 67.0

2.1) Mean day difference for positive cases: 33.0
2.2) Median day difference for positive cases: 34.0

3.1) Mean day difference for all cases: 65.4
3.2) Median day difference for all cases: 50.5


# Inspect "Send Fine"

In [552]:
df_send = df[df['concept_name'].isin(['Send Fine'])]


max_sendfine = (
    df_send.groupby(['case_id', 'concept_name'])['timestamp']
    .agg(['max'])
    .unstack()
    )
max_sendfine.columns = max_sendfine.columns.get_level_values(1)
max_sendfine_dct = max_sendfine.reset_index().set_index('case_id').to_dict()



case_id_max_ts = (
                    df.groupby(['case_id'])['timestamp']
                    .agg(['max'])
                    ).to_dict()


end_with_send_fine = []
for case_id, ts in max_sendfine_dct['Send Fine'].items():
    if case_id_max_ts['max'][case_id] == ts:
        end_with_send_fine.append(case_id)

In [554]:
df.loc[df.case_id==np.random.choice(end_with_send_fine, 1)[0], :]

,amount,resource,dismissal,concept_name,vehicleClass,totalPaymentAmount,lifecycle_transition,timestamp,article,points,case_id,expense,notificationType,lastSent,paymentAmount,matricola
376895,23.0,63,NIL,Create Fine,A,0.0,complete,2010-08-01,7.0,0.0,S155989,NaN,NaN,NaN,NaN,NaN
376896,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2010-09-29,NaN,NaN,S155989,13.5,NaN,NaN,NaN,NaN


In [556]:
print(f"{len(end_with_send_fine)} cases ends with 'Send Fine'")

20755 cases ends with 'Send Fine'


In [558]:
df_ends_with_sendfine = df.loc[df.case_id.isin(end_with_send_fine), :]

In [560]:
store_diffs = find_average_time(df_ends_with_sendfine, 'Create Fine', 'Send Fine', 90)
calculate_stats(store_diffs, 'Create Fine', 'Send Fine', 90)

***** Create Fine(before) & Send Fine(after) ***** Limit: 90 days

1.1) Mean day difference for negative cases: 125.5
1.2) Median day difference for negative cases: 123.0

2.1) Mean day difference for positive cases: 57.0
2.2) Median day difference for positive cases: 58.0

3.1) Mean day difference for all cases: 91.3
3.2) Median day difference for all cases: 90.5


In [562]:
len(store_diffs['positive'])

8813

In [564]:
len(store_diffs['negative'])

11942

In [588]:
more_than_two = pd.DataFrame(df_ends_with_sendfine.groupby('case_id').size()).reset_index().rename(columns={0:'count_row'})

In [601]:
more_than_two_lst = more_than_two.loc[more_than_two.count_row > 2, :].case_id.to_list()

In [634]:
df_ends_with_sendfine.loc[df_ends_with_sendfine.case_id.isin(more_than_two_lst), :].concept_name.unique()

array(['Create Fine', 'Payment', 'Send Fine',
       'Insert Date Appeal to Prefecture', 'Send Appeal to Prefecture',
       'Appeal to Judge'], dtype=object)

pattern is as follows: right after 'Create Fine', if the next activity is either 'payment', 'appeal to judge', or 'Insert date appeal to prefecture', the case ends with 'send fine'.

In [788]:
df_ends_with_sendfine.loc[df_ends_with_sendfine.case_id==np.random.choice(more_than_two_lst, 1)[0], :]

,amount,resource,dismissal,concept_name,vehicleClass,totalPaymentAmount,lifecycle_transition,timestamp,article,points,case_id,expense,notificationType,lastSent,paymentAmount,matricola
83550,65.6,34,NIL,Create Fine,A,0.0,complete,2001-01-28,145.0,0.0,C12875,NaN,NaN,NaN,NaN,NaN
83551,NaN,NaN,NaN,Insert Date Appeal to Prefecture,NaN,NaN,complete,2001-03-14,NaN,NaN,C12875,NaN,NaN,NaN,NaN,NaN
83552,NaN,NaN,#,Send Appeal to Prefecture,NaN,NaN,complete,2001-04-05,NaN,NaN,C12875,NaN,NaN,NaN,NaN,NaN
83553,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2001-05-09,NaN,NaN,C12875,11.88,NaN,NaN,NaN,NaN


##### Cases that include 'Appeal to Judge'

In [22]:
df.loc[(df.concept_name=='Appeal to Judge'), 'case_id'].unique()[:30]

array(['A12414', 'A14727', 'A1516', 'A15307', 'A1582', 'A16141', 'A17158',
       'A17477', 'A17711', 'A18256', 'A18287', 'A18912', 'A19137',
       'A20244', 'A20598', 'A22043', 'A24387', 'A24398', 'A25121',
       'A26902', 'A2725', 'A28368', 'A28617', 'A28691', 'A29455',
       'A30458', 'A31588', 'A32224', 'A34030', 'A35306'], dtype=object)

In [43]:
len(df.loc[(df.concept_name=='Appeal to Judge'), 'case_id'].unique())

555

In [ ]:
#df.loc[(df.case_id == 'A24387'), :]